In [7]:
import pandas as pd

In [8]:
p_data =pd.read_csv('https://raw.githubusercontent.com/PNNL-CompBio/candleDataProcessing/main/cell_line/proteomics.csv.gz')


In [9]:
s_data = pd.read_csv('https://raw.githubusercontent.com/PNNL-CompBio/candleDataProcessing/main/cell_line/samples.csv')


/var/folders/wt/_yr7rg_13t76sq_q5cw57hxw0000gn/T/ipykernel_62173/3184017860.py:1: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  s_data = pd.read_csv('https://raw.githubusercontent.com/PNNL-CompBio/candleDataProcessing/main/cell_line/samples.csv')


In [10]:
g_data = pd.read_csv('https://raw.githubusercontent.com/PNNL-CompBio/candleDataProcessing/main/cell_line/genes.csv')
g_data.rename(columns={"other_id":"ENSG_col"},inplace = True)
g_data = g_data.groupby('ENSG_col').first().reset_index()
g_data = g_data.dropna(subset=['ENSG_col'])

In [11]:
# Merge p_data with s_data to label the rows with other_id
merged = p_data.merge(s_data[['improve_sample_id', 'other_id']], on='improve_sample_id', how='left')
merged = merged.merge(g_data[["entrez_id","gene_symbol","ENSG_col"]], on= "entrez_id",how = "left")

agg_df = merged.groupby(['improve_sample_id', 'entrez_id',"gene_symbol","ENSG_col"]).agg({
    'proteomics': 'mean',
    'other_id': 'first'
}).reset_index()
agg_df.drop(columns=["improve_sample_id"],inplace=True)
agg_df.drop_duplicates(inplace=True)
agg_df = agg_df.dropna(subset=['other_id'])

# Pivot merged dataframe
pivoted = agg_df.pivot(index='other_id', columns=["ENSG_col",'entrez_id',"gene_symbol"], values='proteomics').reset_index()

# Adjust headers
cols = list(pivoted.columns)
cols[0] = ('', '', '')
header = pd.MultiIndex.from_tuples(cols)
pivoted.columns = header

# Remove Duplicates by header
pivoted = pivoted.T
pivoted = pivoted.reset_index().drop_duplicates(subset=['level_1', 'level_2']).set_index(['level_0', 'level_1', 'level_2'])
pivoted = pivoted.T

# Adjust headers again
cols = list(pivoted.columns)
cols[0] = ('', '', '')
header = pd.MultiIndex.from_tuples(cols)
pivoted.columns = header
pivoted

,,ENSG00000175899,ENSG00000171428,ENSG00000127837,ENSG00000090861,ENSG00000183044,ENSG00000165029,ENSG00000107331,ENSG00000167972,ENSG00000131269,...,ENSG00000091138,ENSG00000113211,ENSG00000128000,ENSG00000144802,ENSG00000158485,ENSG00000188064,ENSG00000173610,ENSG00000198018,ENSG00000176566,ENSG00000178343
,,2.0,9.0,14.0,16.0,18.0,19.0,20.0,21.0,22.0,...,1811.0,56130.0,163131.0,64332.0,910.0,7477.0,10941.0,57089.0,138009.0,152573.0
,,A2MD,AAC1,AAMP,AARS,GABA-AT,ABC-1,ABC2,ABC-C,ABC7,...,CLD,PCDH-BETA6,ZNF779,IKBZ,CD1,WNT7B,UDPGT2A1,LALP1,WDR21C,hShisa3
0,ACH-000001,0.285257,-0.838582,-0.230943,-1.766261,0.194806,NaN,NaN,NaN,0.64149,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ACH-000004,-1.172647,0.381142,1.554413,0.864493,-0.32615,0.204095,-1.008918,NaN,-0.838778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ACH-000005,-1.697189,0.111586,1.398722,0.970391,-0.2559,NaN,NaN,NaN,-1.108849,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ACH-000006,-1.106344,1.590329,-0.211486,-0.894781,-0.738681,-0.841487,0.864543,NaN,2.801684,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ACH-000007,-0.903152,1.70815,-0.491505,-0.838761,-1.265735,-1.283112,-0.46095,-0.889231,0.09283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,ACH-000997,-1.594484,0.128803,-0.491202,-0.028403,0.286707,-0.766917,0.119443,0.073544,-0.241875,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,ACH-001198,0.281762,-0.886799,-0.911882,0.480421,0.364107,1.445862,0.923631,0.312197,0.545299,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pivoted.to_csv("proteomics_restructure.tsv", index=False, sep="\t")

In [13]:
pivoted = pivoted.fillna(0) 
pivoted.to_csv("proteomics_restructure_NA_as_zero.tsv", index=False, sep="\t")